import requests
from bs4 import BeautifulSoup as bs
import json
import numpy as np
import pandas as pd
import time
import urllib
import time
import datetime


In [26]:
# df = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/Stations+Typhoon break.csv')
# df.rename({'number': 'ID'}, axis = 1, inplace = True)
# df.drop(df.columns[6:], axis = 1, inplace = True)

In [135]:
# df2 = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_news1.csv', index_col = 'Column1')
# df2.index.name = ''

In [178]:
'''
檢驗typhoon_news_cleansed_1.csv日期格式
通過就就儲存起來
'''
df = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_news_cleansed_1.csv', index_col = 0)
df
def check_date_format(df):
    # 查看年月份格式與[2021-01-01]不同者
    format_wrong_idx = []
    for idx in df2.index:
        for date in df2.loc[idx, 'alert_dates'].split(','):
            if len(date) != 10:
                format_wrong_idx.append(idx)

    # 是否有年月份數字不合理(年或月或日)
    for idx in df2.index:
        for date in df2.loc[idx, 'alert_dates'].split(','):
            yyyy = int(date[:4])
            mm = int(date[5:7])
            dd = int(date[8:])
            if not (yyyy >= 2001 and yyyy <= 2022):
                format_wrong_idx.append(idx)
            if not (mm >= 1 and mm <= 12):
                format_wrong_idx.append(idx)
            if not (dd >= 1 and dd <= 31):
                format_wrong_idx.append(idx)
                
    if len(format_wrong_idx) == 0:
        return('pass')
        print('pass')
    else:
        print('pass-failed')
        for idx in format_wrong_idx:
            print(idx)
        return('pass-failed')

check_date_format(df)
# if check_date_format(df) == 'pass':
#     df2.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_news_cleansed_1.csv')
#     print('pass data saved')
# else:
#     print('pass failed, plz checked data in detail.')

'pass'

In [261]:
'''
Sue 整合的 data ： 將是否放假與station mapping起來，其中日期為wide format
用pd.melt展開成long format
有195筆放假資訊是氣象局發了颱風警報，以及颱風後續造成的影響（水災等等）而有宣布是否放假
當中有155筆放假資訊是氣象局發了颱風警報後宣布放假
'''
df_station_break = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/Stations+Typhoon break_final.csv')
id_var = df_station_break.columns[:6]
value_var = df_station_break.columns[6:]

df_station_break = pd.melt(df_station_break, id_vars=id_var, value_vars=value_var, var_name='issue_dayoff_dates', value_name='day_off')

df_station_break.drop(df_station_break[df_station_break['issue_dayoff_dates'] == '2011-06-12'].index, axis=0, inplace = True)
df_station_break.drop(df_station_break[df_station_break['issue_dayoff_dates'] == '2011-06-13'].index, axis=0, inplace = True)

df_station_break.to_csv('/Users/shyanechang/Desktop/AI_Class/專題/df_station_break.csv')
len(df_station_break.issue_dayoff_dates.value_counts()) # 一共有153天是因為颱風，政府宣布是否放假


153

In [214]:
'''
typhoon_news_cleansed_1.csv當中，發現有重複的日期
將alert_dates中的日期list展開成為date，發現有重複資料 --> 當天有不同颱風的警報
發警戒次數（含重複）共387次
發警戒日期不重複共379個

 (重複日期, 次數)如下:
[('2007-08-08', 2),
 ('2017-07-29', 2),
 ('2017-07-30', 2),
 ('2015-07-09', 2),
 ('2015-07-08', 2),
 ('2010-08-31', 2),
 ('2016-09-15', 2),
 ('2006-08-09', 2)]
'''
df_typhoon = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_news_cleansed_1.csv', header = 0, index_col = 0)

alert_dates = []
for date_list in df_typhoon.alert_dates:  
    alert_dates.extend(date_list.split(','))
# print(len(alert_dates))  # 發警戒次數（含重複）共386次
# print(len(set(alert_dates)))  # 發警戒日期不重複共377個

df_alert_dates = pd.DataFrame({'date': alert_dates}, dtype = 'string')
repeat_times = df_alert_dates.date.value_counts()  # 計算警報日期出現的次數

repeat_dates = []  # 用來儲存重複的日期、重複的次數
for idx in repeat_times.index:
    if repeat_times[idx] > 1:
        repeat_dates.append((idx, repeat_times[idx]))  
    else:
        break
# repeat_dates

# 有發警報但是沒放假的日期整理完之後是0
# 表示放假當天一定有發警報
issue_dayoff_dates = set(df_station_break['issue_dayoff_dates'])
issue_alert_dates = set(alert_dates)

In [217]:
'''
目前有兩個dataframe：

df_station_break（日期已展開）：
包含測站ID與測站的地理資訊以及日期(df_station_break['Date'])  # issue_dayoff_dates
裡面的日期為行政院人事處宣布政令是否因為「颱風」放假的日期，日期不重複（也不會重複）共153筆

df_typhoon（日期未展開）：
包含颱風與日期(alert_dates)  # issue_alert_dates
裡面的日期為發佈警戒的日期，有重複（多個颱風同時侵台時會針對不同颱風發佈警報），不含重複一共379筆

正在做的事：依照上面兩個日期，確認後成為target日期

接下來要做的事：
依照target dates取出yyyy-mm資訊，query條件為MonthDataController，去觀測資料查詢網站下載測站資訊
取得的測站資訊，依照測站id、日期，合併入最後要呈現的dataframe中

後續再做進一步的探索性資料分析、補值等等
'''


"\n目前有兩個dataframe：\n\ndf_station_break（日期已展開）：\n包含測站ID與測站的地理資訊以及日期(df_station_break['Date'])\n裡面的日期為行政院人事處宣布政令是否因為「颱風」放假的日期，日期不重複（也不會重複）共155筆\n\ndf_typhoon（日期未展開）：\n包含颱風與日期(alert_dates)\n裡面的日期為發佈警戒的日期，有重複（多個颱風同時侵台時會針對不同颱風發佈警報），不含重複一共377筆\n\n\n\n接下來要做的事：\n\n確認成為target日期\n\ntarget_dates：\n\n依照target dates取出yyyy-mm資訊，query條件為MonthDataController，去觀測資料查詢網站下載測站資訊\n取得的測站資訊，依照測站id、日期，合併入最後要呈現的dataframe中\n\n後續再做進一步的探索性資料分析、補值等等\n"

In [273]:
'''
2020-08-22巴威颱風是唯一只有發一天警報的，而且沒有放假，可以drop掉
'''
# idxs = []
# for idx, l in zip(df_typhoon.index, df_typhoon['alert_dates'].str.split(',')):
#     if len(l) == 1:
#         idxs.append(idx)
# df_typhoon.loc[idxs[0], :]

ty_name                       巴威
route                         --
alert_dates           2020-08-22
alert_period_start     2020/8/22
alert_period_end       2020/8/22
intensity                     輕度
hpa                          950
wind_speed                    43
7_radius                     220
10_radius                     80
alert_num                      6
land_spot                    NaN
born_spotx                 122.7
born_spoty                  23.3
Name: 6, dtype: object

In [318]:
'''
將df_typhoon flatten
'''
typhoon_new_dict = {
    'ty_name':[], 
    'route': [], 
    'issue_alert_dates': [], 
    'intensity': [], 
    'hpa': [], 
    'wind_speed': [], 
    '7_radius': [], 
    '10_radius': [], 
    'alert_num': [], 
    'born_spotx': [], 
    'born_spoty': [],
}

keys = list(typhoon_new_dict.keys())
keys.remove('issue_alert_dates')

for idx in df_typhoon.index:
    typhoon_new_dict['issue_alert_dates'].extend([date for date in df_typhoon.loc[idx, 'alert_dates']])
    for key in keys:
        typhoon_new_dict[key].extend([df_typhoon.loc[idx,key]]*len(df_typhoon.loc[idx, 'alert_dates']))

df_typhoon_flattened = pd.DataFrame(typhoon_new_dict)



# df_typhoon_new.merge(df_typhoon, on = 'ty_name', validate = 'many_to_one')
# df_typhoon
# for idx in df_typhoon.index:
#     df_typhoon.loc[idx, 'ty']

In [323]:
# df_typhoon_flattened.to_csv('df_typhoon_flattened.csv', encoding = 'utf-8')

In [341]:
station_alert_dates = pd.DataFrame([
    df_,
    df_typhoon_flattened['issue_alert_dates']
])

,ty_name,route,issue_alert_dates,intensity,hpa,wind_speed,7_radius,10_radius,alert_num,born_spotx,born_spoty
0,圓規,--,2021-10-10,輕度,975,30,250,80,13,131.8,12.7
1,圓規,--,2021-10-11,輕度,975,30,250,80,13,131.8,12.7
2,圓規,--,2021-10-12,輕度,975,30,250,80,13,131.8,12.7
3,璨樹,6,2021-09-10,強烈,915,58,200,70,24,136.6,15.6
4,璨樹,6,2021-09-11,強烈,915,58,200,70,24,136.6,15.6
...,...,...,...,...,...,...,...,...,...,...,...
382,奇比,7,2001-06-23,中度,965,35,200,80,19,133.0,13.0
383,奇比,7,2001-06-24,中度,965,35,200,80,19,133.0,13.0
384,西馬隆,8,2001-05-11,輕度,990,23,150,--,20,119.3,13.8
385,西馬隆,8,2001-05-12,輕度,990,23,150,--,20,119.3,13.8


In [382]:
df_typhoon_flattened['issue_alert_dates'].str[:7].unique()

# df_station_break.Station.nunique()

array(['2021-10', '2021-09', '2021-08', '2021-07', '2021-06', '2020-11',
       '2020-08', '2020-05', '2019-09', '2019-10', '2019-08', '2019-07',
       '2018-09', '2018-07', '2017-09', '2017-08', '2017-07', '2016-10',
       '2016-09', '2016-07', '2015-09', '2015-08', '2015-07', '2015-05',
       '2014-09', '2014-07', '2014-06', '2013-10', '2013-09', '2013-08',
       '2013-07', '2012-09', '2012-08', '2012-07', '2012-06', '2011-08',
       '2011-06', '2011-05', '2010-10', '2010-09', '2010-08', '2009-10',
       '2009-08', '2009-07', '2009-06', '2008-09', '2008-08', '2008-07',
       '2007-11', '2007-10', '2007-09', '2007-08', '2006-09', '2006-08',
       '2006-07', '2006-05', '2005-09', '2005-10', '2005-08', '2005-07',
       '2004-12', '2004-10', '2004-09', '2004-08', '2004-07', '2004-06',
       '2003-11', '2003-08', '2003-09', '2003-07', '2003-06', '2003-04',
       '2002-09', '2002-07', '2007-07', '2001-10', '2001-09', '2001-07',
       '2001-06', '2001-05'], dtype=object)

In [38]:
df_typhoon_flattened = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_info/df_typhoon_flattened.csv', index_col = 0)
df_station = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/Stations+Typhoon break_final.csv')
df_station_existing = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/existing_station.csv')


In [39]:
# Sue整理有，但網路上的沒有
set(df_station['number']) - set(df_station_existing['站號'])
df_station['number'].duplicated().sum()  # Sue整理的測站編號沒有沒有重複

len(df_station['number'])  # Sue整理的現存站點共573個

# Sue整理的站名(Station)有8筆重複，每筆重複個一次
# 重複的['關山', '石牌', '大坑', '竹塘', '五分山', '東河', '信義', '復興']
station_name1 = df_station['Station'].value_counts().head(8).index


In [40]:
# 網路上有，但Sue整理的沒有
set(df_station_existing['站號']) - set(df_station['number'])
df_station_existing['站號'].duplicated().sum()  # CWB網站上的測站編號也沒有重複
len(df_station_existing) # 網路上現存站點共618個

# 網路上的站名(站名)有5筆重複，每筆重複個一次
# 重複的['復興', '信義', '東河', '大坑', '關山']
station_name2 = df_station_existing['站名'].value_counts().head(5).index

In [48]:
# 主要是要看明珩的站名(Station)有沒有無效的重複
station_diff = [i for i in (set(station_name1) - set(station_name2))]
# df_station['Station'] == ''
station_diff

['五分山', '石牌', '竹塘']

In [54]:
# idx = sum([],[])
import time
# idx = [index for index in df_station[df_station['Station'] == s].index for s in station_diff]

start1 = time.process_time()
idx = []
for s in station_diff:
    idx.extend(df_station[df_station['Station'] == s].index)
end1 = time.process_time()


start2 = time.process_time()
idx_ = []
for s in station_diff:
    for index in df_station[df_station['Station'] == s].index:
        idx_.append(index)
end2 = time.process_time()
print('time1: ', end1-start1)
print('time2: ', end2-start2)
print(idx)
print(idx_)

time1:  0.002177999999999791
time2:  0.0019179999999998643
[24, 54, 14, 15, 177, 184]
[24, 54, 14, 15, 177, 184]


In [55]:
l = [[1,2,3], [4,5,6], [1,3,2,4,5]]


In [471]:
df_station_existing[df_station_existing['站號'] == str(466850)]
df_station[df_station['Station'] == '墾丁雷達站']

,Region,Station,number,altitude,longitude,region,2001-07-30,2001-07-31,2001-09-17,2001-09-18,...,2021-06-04,2021-07-22,2021-07-23,2021-07-24,2021-08-05,2021-09-11,2021-09-12,2021-09-13,2021-10-11,2021-10-12
373,屏東縣,墾丁雷達站,467790,21.9028,120.8469,恆春鎮,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [26]:
s1 = {
    'a': [1,2,3,4,5,6,1,2,3,4],
    'b': [i for i in range(100, 110)]
}
df1 = pd.DataFrame(s1)

s2 = {
    'a': [1,2,3,4,5,6,1,2,3,4],
    'b': [i for i in range(100, 110)]
}
df2 = pd.DataFrame(s1)

s3 = {
    'a': [1,2,3,4,5,6,1,2,3,4],
    'b': [i for i in range(100, 110)]
}
df3 = pd.DataFrame(s1)

dfs = {
    'df_1':df1,
    'df_2':df2,
    'df_3':df3,
}

In [49]:
df_station

,Region,Station,number,altitude,longitude,region,2001-07-30,2001-07-31,2001-09-17,2001-09-18,...,2021-06-04,2021-07-22,2021-07-23,2021-07-24,2021-08-05,2021-09-11,2021-09-12,2021-09-13,2021-10-11,2021-10-12
0,基隆市,基隆,466940,25.1351,121.7323,中山區,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
1,基隆市,彭佳嶼,466950,25.6298,122.0715,中正區,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
2,基隆市,七堵,C0B010,25.0966,121.7089,七堵區,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
3,基隆市,基隆嶼,C0B020,25.1932,121.7757,中正區,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
4,臺北市,臺北,466920,25.0394,121.5067,中正區,1,0,1,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,連江縣,東引,C0W170,26.3605,120.4753,東引鄉,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
569,金門縣,金門,467110,24.4089,118.2811,金城鎮,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
570,金門縣,金沙,C0W140,24.4899,118.4006,金沙鎮,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
571,金門縣,烏坵,C0W160,24.9961,119.4413,金沙鎮,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
df_typhoon_flattened = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/typhoon_info/df_typhoon_flattened.csv', index_col = 0)
df_station = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/Stations+Typhoon break_final.csv')
df_station_existing = pd.read_csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_info/existing_station.csv')
df_observations = pd.read.csv('/Users/shyanechang/Desktop/AI_Class/專題/data/station_observation/observations.csv')

In [ ]:
df_observations